In [2]:
import requests
import requests_ftp
import requests_cache
import lxml
from lxml import html
from bs4 import BeautifulSoup
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd
import urllib
import json
import cloudscraper
import sqlalchemy as sqla

In [3]:
engine = sqla.create_engine('sqlite:///top200data')
url = "https://spotifycharts.com/regional/global/daily/2021-05-25"
scraper = cloudscraper.create_scraper()
r = scraper.get("https://spotifycharts.com/regional/global/daily/2021-05-25")
clim = BeautifulSoup(r.text,'lxml')

In [4]:
clim


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Spotify Charts</title>
<meta content="Spotify Charts" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://www.spotifycharts.com/" property="og:url"/>
<meta content="http://d2c87l0yth4zbw-2.global.ssl.fastly.net/i/_global/open-graph-default.png" property="og:image"/>
<meta content="Spotify is all the music you’ll ever need." property="og:description"/>
<meta content="Spotify is all the music you’ll ever need." name="description"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/images/touch-icon-144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/touch-icon-114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/touch-icon-72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/touch-icon-57.png" rel="apple-touch

In [5]:
regiondata=pd.DataFrame(columns=['code','country'])

for i in range(70):
    regiondata.loc[i,"code"]=clim.find_all(name='li')[:70][i].get('data-value')
    regiondata.loc[i,"country"]=clim.find_all(name='li')[:70][i].text
regiondata

,code,country
0,global,Global
1,us,United States
2,gb,United Kingdom
3,ae,United Arab Emirates
4,ar,Argentina
...,...,...
65,tw,Taiwan
66,ua,Ukraine
67,uy,Uruguay
68,vn,Viet Nam


In [270]:
url = "https://spotifycharts.com/regional/{}/daily/2021-05-25".format(regiondata.loc[1,"code"])
url

'https://spotifycharts.com/regional/us/daily/2021-05-25'

In [9]:
rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
link = clim.find_all(name='a')[6:-5] #only song links
info[0].text.splitlines()[2][3:]
link[199].get('href')[-22:]

'6RUKPb4LETWmmr3iAEQktW'

In [15]:
info[0].text.splitlines()


['', 'good 4 u', 'by Olivia Rodrigo']

In [169]:
id = "date_{}_region".format(int(rank[0].text))
filler = (id, int(rank[0].text), info[0].text.splitlines()[1] , "date" , info[0].text.splitlines()[2][3:], "region")
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'PRIMARY KEY (id));')
engine.execute('INSERT INTO "top200" '
               '(id, rank, song, date, artist,region) '
               'VALUES (?,?,?,?,?,?) ', filler )
pd.read_sql('select * from top200',engine)

,id,rank,song,date,artist,region
0,date_1_region,1,good 4 u,date,Olivia Rodrigo,region


In [209]:
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'songcode VARCHAR, '
               'PRIMARY KEY (id));')


for i in range(200):
    id = "date_{}_region".format(int(rank[i].text))
    filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], "region", link[i].get('href')[-22:])

    engine.execute('INSERT INTO "top200" '
                   '(id, rank, song, date, artist,region, songcode) '
                   'VALUES (?,?,?,?,?,?,?) ', filler )
pd.read_sql('select * from top200',engine)


,id,rank,song,date,artist,region,songcode
0,date_1_region,1,good 4 u,date,Olivia Rodrigo,region,4ZtFanR9U6ndgddUvNcjcG
1,date_2_region,2,Butter,date,BTS,region,3VqeTFIvhxu3DIe4eZVzGq
2,date_3_region,3,deja vu,date,Olivia Rodrigo,region,6HU7h9RYOaPRFeh0R3UeAr
3,date_4_region,4,MONTERO (Call Me By Your Name),date,Lil Nas X,region,67BtfxlNbhBmCDR2L2l8qd
4,date_5_region,5,Kiss Me More (feat. SZA),date,Doja Cat,region,748mdHapucXQri7IAO8yFK
...,...,...,...,...,...,...,...
195,date_196_region,196,Savage Love (Laxed - Siren Beat),date,"Jawsh 685, Jason Derulo",region,1xQ6trAsedVPCdbtDAmk0c
196,date_197_region,197,La Nota,date,"Manuel Turizo, Rauw Alejandro, Myke Towers",region,5VzNsUnLmYcRfRfxpQsusa
197,date_198_region,198,If the World Was Ending - feat. Julia Michaels,date,JP Saxe,region,2kJwzbxV2ppxnQoYw4GLBZ
198,date_199_region,199,Somebody That I Used To Know,date,"Gotye, Kimbra",region,1qDrWA6lyx8cLECdZE7TV7


In [300]:
engine.execute('DROP TABLE IF EXISTS "top200";')
engine.execute('CREATE TABLE "top200" ('
               'id VARCHAR,'
               'rank INTEGER NOT NULL, '
               'song VARCHAR, '
               'date VARCHAR, '
               'artist VARCHAR, '
               'region VARCHAR, '
               'regioncode VARCHAR, '
               'songcode VARCHAR, '
               'PRIMARY KEY (id));')

scraper = cloudscraper.create_scraper()
for place in range(70):


    url = "https://spotifycharts.com/regional/{}/daily/2021-05-25".format(regiondata.loc[place,'code'])

    r = scraper.get(url)
    clim = BeautifulSoup(r.text,'lxml')
    
    rank = clim.find_all(name='td',attrs={'class':'chart-table-position'})
    info = clim.find_all(name='td',attrs={'class':'chart-table-track'})
    link = clim.find_all(name='a')[6:-5] #only song links
    
    for i in range(len(rank)):
        id = "date_{}_{}".format(int(rank[i].text),place)
        filler = (id, int(rank[i].text), info[i].text.splitlines()[1] , "date" , info[i].text.splitlines()[2][3:], regiondata.loc[place,'country'], regiondata.loc[place,'code'], link[i].get('href')[-22:])

        engine.execute('INSERT INTO "top200" '
                       '(id, rank, song, date, artist, region, regioncode, songcode) '
                   'VALUES (?,?,?,?,?,?,?,?) ', filler )
pd.read_sql('select * from top200',engine)


,id,rank,song,date,artist,region,regioncode,songcode
0,date_1_0,1,good 4 u,date,Olivia Rodrigo,Global,global,4ZtFanR9U6ndgddUvNcjcG
1,date_2_0,2,Butter,date,BTS,Global,global,3VqeTFIvhxu3DIe4eZVzGq
2,date_3_0,3,deja vu,date,Olivia Rodrigo,Global,global,6HU7h9RYOaPRFeh0R3UeAr
3,date_4_0,4,MONTERO (Call Me By Your Name),date,Lil Nas X,Global,global,67BtfxlNbhBmCDR2L2l8qd
4,date_5_0,5,Kiss Me More (feat. SZA),date,Doja Cat,Global,global,748mdHapucXQri7IAO8yFK
...,...,...,...,...,...,...,...,...
13286,date_196_69,196,All Girls Are The Same,date,Juice WRLD,South Africa,za,4VXIryQMWpIdGgYR4TrjT1
13287,date_197_69,197,Stripes,date,"Blxckie, Flvme",South Africa,za,1uVpFlnT4n8JSPbvG32DLe
13288,date_198_69,198,Ina Iyeza,date,Zain SA,South Africa,za,2VeRkDTAwqGFKrGXQnsWEY
13289,date_199_69,199,Something Just Like This,date,"The Chainsmokers, Coldplay",South Africa,za,6RUKPb4LETWmmr3iAEQktW


In [105]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='3b8a7beb4a434db98620cfae1983cffc',client_secret='a5bc3f14c035496690b540b81c5053f5',))

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])


Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [171]:
len('4ZtFanR9U6ndgddUvNcjcG')

22

In [288]:
test=pd.DataFrame(columns=['test'])
for place in range(70):
    
    for i in range(200):
        
        print("date_{}_{}".format(int(rank[i].text),place))
        

In [289]:
a

14000

id                        date_130_8
rank                             130
song                         Beggin'
date                            date
artist                      Måneskin
region                      Bulgaria
regioncode                        bg
songcode      3Wrjm47oTz2sjIgck11l5e
Name: 129, dtype: object

In [302]:
pd.read_sql('select * from top200',engine).to_csv("top200_20210525.csv")